# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LORA
* Model: GPT2
* Evaluation approach: I decided to use something similar to the Bert sentient classifier we learned about in the course, but in relation to objectivity and fact. I found another model on huggingface that was trained for fact/objectivity or subjectivity/opinion, and I found it interesting to classify it as a sequence, referenced here: https://huggingface.co/lighteternal/fact-or-opinion-xlmr-el
* Fine-tuning dataset: I used the "nyu-mll/glue" dataset that met CoLA GLUE benchmark: https://huggingface.co/datasets/nyu-mll/glue 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM
from datasets import load_dataset
import torch

tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForSequenceClassification.from_pretrained('gpt2',
                                                      #num_labels = 2,
                                                      #id2label = {0: "Opinion/Subjective sentence", 1:"Fact/Objective sentence"},
                                                      #label2id = {"Opinion/Subjective sentence":0, "Fact/Objective sentence":1}
                                                      )

dataset =  load_dataset("nyu-mll/glue", 'cola')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [2]:
input = tokenizer("Food is scrappy", return_tensors = 'pt')

print("Test Sentence: food is scrappy")

with torch.no_grad():
    outputs = model(**input).logits
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    predicted_class = torch.argmax(probabilities)

if predicted_class == 1:
    print(f"\nThere is a {probabilities[0][0] * 100:.2f}% chance that it is a Fact/Objective sentence")
else:
    print(f"\nThere is a {probabilities[0][0] * 100:.2f}% chance that it is an Opinion/Subjective sentence")
    
input = tokenizer(dataset['train'][1]['sentence'], return_tensors = 'pt')

print("Test Sentence: ",dataset['train'][1]['sentence'])

with torch.no_grad():
    outputs = model(**input).logits
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    predicted_class = torch.argmax(probabilities)

if predicted_class == 1:
    print(f"\nThere is a {probabilities[0][0] * 100:.2f}% chance that it is a Fact/Objective sentence")
else:
    print(f"\nThere is a {probabilities[0][0] * 100:.2f}% chance that it is an Opinion/Subjective sentence")

Test Sentence: food is scrappy

There is a 100.00% chance that it is an Opinion/Subjective sentence
Test Sentence:  One more pseudo generalization and I'm giving up.

There is a 100.00% chance that it is an Opinion/Subjective sentence


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [3]:
dataset['train'] = dataset['train'].select(range(500))
dataset['test'] = dataset['test'].select(range(500))

def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding = True, truncation=True, return_tensors = "pt")
tokenized_ds = {}


tokenized_ds['train'] = dataset['train'].map(tokenize_function, batched=True)
tokenized_ds['test'] = dataset['test'].map(tokenize_function, batched=True)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [4]:
from peft import LoraConfig
from peft import get_peft_model


config = LoraConfig(task_type="SEQ_CLS",
                   r = 8,
                   lora_dropout=0.01,
                   )

lora_model = get_peft_model(model, config)
# used link below to fix issue with batch > -1 =, used eos_toke_id
# https://stackoverflow.com/questions/68084302/assertionerror-cannot-handle-batch-sizes-1-if-no-padding-token-is-defined
lora_model.config.pad_token_id = lora_model.config.eos_token_id 

for param in lora_model.parameters():
    param.requires_grade = False

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [5]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = {"accuracy": (predictions == labels).mean()}
    return accuracy

trainer_model = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data_save",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=1,
        weight_decay=0.01,
        #changed save and evaluation strategy by referencing this article 
        #recommended by the tech help section in our circle community:
        # https://community.udacity.com/c/externship-tech-help/train-peft-model-bug-i-can-train-my-model-and
        save_strategy="epoch", 
        evaluation_strategy="epoch", 
        load_best_model_at_end = True,
        remove_unused_columns = True
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, return_tensors = "pt"),
    compute_metrics=compute_metrics,
   
)

trainer_model.train()
model.save_pretrained("pre_trained")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.984211,0.658000


Checkpoint destination directory ./data_save/checkpoint-250 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [6]:

import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer_lora = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data_save",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=1,
        weight_decay=0.01,
        #changed save and evaluation strategy by referencing this article 
        #recommended by the tech help section in our circle community:
        # https://community.udacity.com/c/externship-tech-help/train-peft-model-bug-i-can-train-my-model-and
        save_strategy="epoch", 
        evaluation_strategy="epoch", 
        load_best_model_at_end = True,
        remove_unused_columns = True
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, return_tensors = "pt"),
    compute_metrics=compute_metrics,
   
)

trainer_lora.train()

lora_model.save_pretrained("gpt-lora")

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.797381,0.658000


Checkpoint destination directory ./data_save/checkpoint-250 already exists and is non-empty.Saving will proceed but saved results may be invalid.


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [7]:
from peft import AutoPeftModelForSequenceClassification
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora")
model = AutoModelForSequenceClassification.from_pretrained("pre_trained")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at pre_trained were not used when initializing GPT2ForSequenceClassification: ['transformer.h.6.attn.c_attn.lora_A.default.weight', 'transformer.h.9.attn.c_attn.lora_A.default.weight', 'transformer.h.5.attn.c_attn.lora_A.default.weight', 'transformer.h.7.attn.c_attn.lora_A.default.weight', 'transformer.h.5.attn.c_attn.lora_B.default.weight', 'score.original_module.weight', 'transformer.h.7.attn.c_attn.lora_B.default.weight', 'transformer.h.11.attn.c_attn.lora_B.default.weight', 'transformer.h.2.attn.c_attn.lora_B.default.weight', 'transformer.h.3.attn.c_attn.lora_A.default.weight', 'transformer.h.10.attn.c_attn.lora_A.default.weight', 'transformer.h.1.attn.c_attn.lora_B.default.weight', 'transforme

In [8]:
eval_lora = trainer_lora.evaluate()
eval_trainer = trainer_model.evaluate()
print("The accuracy of the pretrained model is:", eval_trainer['eval_accuracy'])
print("The accuracy of the lora model is:" ,eval_lora['eval_accuracy'])

The accuracy of the pretrained model is: 0.658
The accuracy of the lora model is: 0.658
